In [1]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from sklearn.model_selection import train_test_split

In [2]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)

In [3]:
N_EPOCHS = 50
BATCH_SIZE = 512

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [6]:
x, y = utils.load_cifar_data("../data/cifar-10-batches-py/")
x /= 255

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]

In [7]:
train_size, len(x_val), len(x_test)

(30000, 10000, 10000)

In [8]:
assessment_model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

history = assessment_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/50


2024-03-27 15:22:41.546223: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-03-27 15:22:41.546250: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-03-27 15:22:41.546256: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-03-27 15:22:41.546274: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-27 15:22:41.546287: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-03-27 15:22:42.391379: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.1447 - loss: 2.3837 - val_accuracy: 0.2016 - val_loss: 2.1571
Epoch 2/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.2367 - loss: 2.0549 - val_accuracy: 0.1922 - val_loss: 2.1129
Epoch 3/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.2897 - loss: 1.9251 - val_accuracy: 0.1879 - val_loss: 2.1444
Epoch 4/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3293 - loss: 1.8317 - val_accuracy: 0.2148 - val_loss: 2.1132
Epoch 5/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3496 - loss: 1.7536 - val_accuracy: 0.2235 - val_loss: 2.1058
Epoch 6/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3706 - loss: 1.7039 - val_accuracy: 0.2802 - val_loss: 1.9289
Epoch 7/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3747 - loss: 1.6726 - val_accuracy: 0.3467 - val_loss: 1.7669
Epoch 8/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3972 - loss: 1.6234 - val_accuracy: 0.4100 - val_loss: 1.

In [9]:
assessment_model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5315 - loss: 1.3379


[1.3303395509719849, 0.531499981880188]

In [10]:
assessment_model.save_weights('../models/assessment_model.weights.h5')